In [26]:
import pandas as pd
commodity = 'corn'
import utils
import numpy as np

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [163]:
import dataloader
import LSTM

In [171]:
train_loader = dataloader.get_dataloader('corn', 20, 'train', 10)
validation_loader = dataloader.get_dataloader('corn', 20, 'val', 20)

In [174]:
model = LSTM.CommodityLSTM()

In [ ]:
model.train(train_loader, validation_loader)

TRAINING BATCH LOSS AT STEP 0: 0.21180841326713562
VALIDATION BATCH LOSS AT STEP 0: 0.20851445198059082
TRAINING BATCH LOSS AT STEP 1: 0.21229520440101624
VALIDATION BATCH LOSS AT STEP 1: 0.20625078678131104
TRAINING BATCH LOSS AT STEP 2: 0.20882990956306458
VALIDATION BATCH LOSS AT STEP 2: 0.2039884626865387
TRAINING BATCH LOSS AT STEP 3: 0.21214628219604492
VALIDATION BATCH LOSS AT STEP 3: 0.20172683894634247
TRAINING BATCH LOSS AT STEP 4: 0.19908134639263153
VALIDATION BATCH LOSS AT STEP 4: 0.1994677484035492
TRAINING BATCH LOSS AT STEP 5: 0.20198872685432434
VALIDATION BATCH LOSS AT STEP 5: 0.1972099393606186
TRAINING BATCH LOSS AT STEP 6: 0.19553595781326294
VALIDATION BATCH LOSS AT STEP 6: 0.19495435059070587
TRAINING BATCH LOSS AT STEP 7: 0.19668325781822205
VALIDATION BATCH LOSS AT STEP 7: 0.19270016252994537
